In [79]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive
!pip install tqdm ipdb

gpg: keybox '/tmp/tmplmv8qrim/pubring.gpg' created
gpg: /tmp/tmplmv8qrim/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


KeyboardInterrupt: ignored

In [2]:
# !umount drive
# !google-drive-ocamlfuse drive
# !ls drive

In [0]:
import os
import pickle
import numpy as np
import keras
import matplotlib.pyplot as plt
import time
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import ipdb  # remove this 
from tqdm import tqdm
from collections import Counter
from itertools import chain

In [0]:
SEED = 42

# this is the vocab size for 100d vectors in Glove according to details provided by Staford NLP group here https://nlp.stanford.edu/projects/glove/ 
VOCAB_SIZE = 40000
EMBEDDING_DIM = 100 # it's possible to try different dimensions here 

DATA_FOLDER = "drive/text_summarization/refactored"
OLD_DATA_FOLDER = "drive/text_summarization"
INPUT_DATA_FILENAME = "cleaned_train_data.json"
OUTPUT_DATA_FILENAME = "smaller_desc_dataset.pkl"
OUTPUT_DATA_TOKENIZED_FILENAME = "tokens_1M.pkl"
GLOVE_WEIGHTS_FILENAME = "glove.6B.{}d.txt".format(EMBEDDING_DIM)
VOCABULARY_EMBEDDINGS_FILENAME = "vocabulary-embedding.pkl"
VOCABULARY_EMBEDDINGS_DATA_FILENAME = "vocabulary-embedding_data.pkl"

EMPTY = 0 # RNN mask of no data
EOS = 1  # end of sentence
START_IDX = EOS + 1 # first real word

GLOVE_SIZE = 400000  # XXL
GLOVE_DIST_THRESHOLD = 0.5
UNKNOWN_WORDS_NUMBER = 100

In [0]:
%matplotlib inline
np.random.seed(SEED)

In [0]:
def tokenize_sentence(sentence):
    """Tokenize a sentence."""
    return ' '.join(list(nltk.word_tokenize(sentence)))
  

def tokenize_text(text):
    """Tokenize all sentences in text."""
    tokenized = []
    for r in tqdm(text):
        tokenized.append(tokenize_sentence(r))
    return tokenized
  
  
def get_vocab_data(topics):
    vocab_words_count = Counter(word for sentence in topics for word in sentence.split())
    vocab = list(map(lambda entry: entry[0], 
                     sorted(vocab_words_count.items(), key=lambda entry: -entry[1])))
    return vocab, vocab_words_count
  
  
def get_idx(vocab, empty_symbol, eos_symbol):
    word2idx = dict((word, idx + START_IDX) for idx, word in enumerate(vocab))
    word2idx['<empty>'] = empty_symbol
    word2idx['<eos>'] = eos_symbol
    
    idx2word = dict((idx, word) for word, idx in word2idx.items())

    return word2idx, idx2word

In [0]:
data = pd.read_json(open(os.path.join(OLD_DATA_FOLDER, INPUT_DATA_FILENAME)))   # replace OLD_DATA_FOLDER

In [0]:
# extracting only first N sentences from the descriptions otherwise we wil process a lot of unnecessary data
max_sentences_in_description = 2

# addind "." in the end of each sentence as this is the sign of end of sentence
# it can be done with nltk.tokenize.sent_tokenize, but this process never ended, so I did it this way
data['content'] = [".".join(full_desc.split(".")[:max_sentences_in_description]) + "." for full_desc in data['content']]

In [0]:
with open(os.path.join(DATA_FOLDER, OUTPUT_DATA_FILENAME), 'wb') as fp:
    pickle.dump(data, fp, -1)

In [8]:
# tokenizing the data
nltk.download('punkt')
tokenized_result = data.apply(tokenize_text, axis = 0)

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


100%|██████████| 1000000/1000000 [08:59<00:00, 1853.96it/s]


100%|██████████| 1000000/1000000 [09:05<00:00, 1834.48it/s]


100%|██████████| 1000000/1000000 [02:32<00:00, 6543.29it/s]


In [0]:
with open(os.path.join(DATA_FOLDER, OUTPUT_DATA_TOKENIZED_FILENAME), 'wb') as f:
    pickle.dump(tokenized_result, f) 

In [0]:
# when uncommenting one row - comment other. 
# depends whether you are running a whole pipeline or want to use serialized data


data = pd.read_pickle(open(os.path.join(DATA_FOLDER, OUTPUT_DATA_TOKENIZED_FILENAME), 'rb'))  
# data = tokenized_result

In [0]:
heads = data['title']
descriptions = data['content']

vocab, vocab_word_count = get_vocab_data(heads + descriptions)

In [0]:
word2idx, idx2word = get_idx(vocab, EMPTY, EOS)

In [0]:
# source - http://nlp.stanford.edu/data/glove.6B.zip
glove_weights_filepath = os.path.join(DATA_FOLDER, GLOVE_WEIGHTS_FILENAME)

In [0]:
glove_index_dict = {}
glove_embedding_weights = np.empty((GLOVE_SIZE, EMBEDDING_DIM))
globale_scale = 0.1

with open(glove_weights_filepath, 'r') as f:
    for idx, line in enumerate(f.readlines()):
        line = line.strip().split()
        word, vector = line[0], line[1:]
        glove_index_dict[word] = idx
        glove_embedding_weights[idx] = np.array(list(map(float, vector)))
        
        #  adding lowercased versions of uppercased words with the same index 
        word_lowered = word.lower()
        if word_lowered != word:
            glove_index_dict[word_lowered] = idx
        
glove_embedding_weights *= globale_scale

In [0]:
# uniformly generate random embeddings in range (mean - 3std, mean + 3std) 
# (using rule of three deviations here) 
glove_std = glove_embedding_weights.std()
glove_mean = glove_embedding_weights.mean()
embeddings = np.random.uniform(low=glove_mean - 3 * glove_std, 
                               high=glove_mean + 3 * glove_std,  
                               size=(VOCAB_SIZE, EMBEDDING_DIM))

In [0]:
# copy embeddings for most popular words from our vocabulary 
# from glove to matrix created in previous cell
for idx in range(VOCAB_SIZE):
    our_word = idx2word[idx]
    glove_idx = glove_index_dict.get(our_word, glove_index_dict.get(our_word.lower()))
    if glove_word is not None:
        embeddings[idx] = glove_embedding_weights[glove_idx]

In [0]:
word2glove = {}

# store words from our vocabulary which are present in glove
for word in word2idx:
    if word in glove_index_dict:
        glove_word = word
    elif word.lower() in glove_index_dict:
        glove_word = word.lower()
    else:
        continue
    word2glove[word] = glove_word

In [28]:
normed_embedding = embeddings / np.array([np.sqrt(np.dot(weight, weight)) for weight in embeddings])[:, None]

glove_match = []

for our_word, idx in tqdm(word2idx.items()):
    if idx >= VOCAB_SIZE - UNKNOWN_WORDS_NUMBER and our_word.isalpha() and our_word in word2glove:
        glove_emb = glove_embedding_weights[glove_index_dict[word2glove[our_word]]].copy()
        glove_emb /= np.sqrt(np.dot(glove_emb, glove_emb))
        score = np.dot(normed_embedding[:VOCAB_SIZE - UNKNOWN_WORDS_NUMBER], glove_emb)
        while True:
            emb_idx = score.argmax()
            dist = score[emb_idx]
            if dist < GLOVE_DIST_THRESHOLD:
                break
                
            if idx2word[emb_idx] in word2idx:
                glove_match.append((our_word, emb_idx, dist)) 
                break
            score[emb_idx] = -1
            
glove_match.sort(key = lambda x: -x[2]) # sort by distance


  3%|▎         | 47334/1470841 [00:17<08:47, 2700.32it/s]


  3%|▎         | 48413/1470841 [00:21<10:27, 2266.06it/s]


  3%|▎         | 49437/1470841 [00:25<12:07, 1954.58it/s]


  3%|▎         | 50577/1470841 [00:29<13:37, 1737.07it/s]


  4%|▎         | 51772/1470841 [00:33<15:05, 1566.78it/s]

...

 95%|█████████▍| 1395428/1470841 [10:29<00:34, 2216.48it/s]


 96%|█████████▌| 1406685/1470841 [10:33<00:28, 2221.06it/s]


 97%|█████████▋| 1426921/1470841 [10:37<00:19, 2239.67it/s]


 99%|█████████▉| 1454092/1470841 [10:40<00:07, 2269.28it/s]


100%|██████████| 1470841/1470841 [10:43<00:00, 2286.70it/s]

In [0]:
glove_idx2idx = dict((word2idx[w], embedding_idx) 
                     for  w, embedding_idx, _ in glove_match)

In [0]:
heads_indices = [
    [word2idx[word] if word in word2idx.keys() else EMPTY for word in headline.split()] 
    for headline in heads]

In [0]:
descriptions_indices = [
    [word2idx[word] if word in word2idx.keys() else EMPTY for word in description.split()] 
    for description in descriptions]

In [0]:
with open(os.path.join(DATA_FOLDER, VOCABULARY_EMBEDDINGS_FILENAME), 'wb') as fp:
    pickle.dump((embeddings, idx2word, word2idx, glove_idx2idx), fp, -1)

In [0]:
with open(os.path.join(DATA_FOLDER, VOCABULARY_EMBEDDINGS_DATA_FILENAME), 'wb') as fp:
    pickle.dump((descriptions_indices, heads_indices), fp, -1)